In [5]:
import sys
!{sys.executable} -m ensurepip --upgrade
!{sys.executable} -m pip install PyPDF2

Looking in links: c:\Users\shuyu\AppData\Local\Temp\tmp1qetqr0m
Processing c:\users\shuyu\appdata\local\temp\tmp1qetqr0m\pip-25.2-py3-none-any.whl
  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)


In [6]:
from typing import List
from sentence_transformers import SentenceTransformer
import chromadb
from sentence_transformers import CrossEncoder
from dotenv import load_dotenv
from google import genai
import math
import PyPDF2

load_dotenv()
google_client = genai.Client()

chromadb_client = chromadb.EphemeralClient()
chromadb_collection = chromadb_client.get_or_create_collection(name="default")


embedding_model = SentenceTransformer("shibing624/text2vec-base-chinese")


In [21]:
import os

In [19]:
def split_into_chunks(doc_file: str) -> List[str]:
    file_extension = os.path.splitext(doc_file)[1].lower()
    if file_extension == '.pdf':
        return split_pdf_into_chunks(doc_file)
    else:
        with open(doc_file, 'r', encoding = 'utf-8') as file:
            content = file.read()
        return [chunk for chunk in content.split("\n\n")]


In [9]:
def split_pdf_into_chunks(pdf_file: str) -> List[str]:
    """专门处理PDF文件的函数"""
    chunks = []
    with open(pdf_file, 'rb') as file:  # 注意使用二进制模式
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            text = page.extract_text()
            # 按两个换行符分割文本
            page_chunks = [chunk for chunk in text.split("\n\n") if chunk.strip()]
            chunks.extend(page_chunks)
    return chunks

In [10]:
def embed_chunk(chunk: str) -> List[float]:
    embedding = embedding_model.encode(chunk, normalize_embeddings=True)
    return embedding.tolist()

In [11]:
def save_embeddings(chunks: List[str], embeddings: List[List[float]]) -> None:
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        chromadb_collection.add(
            documents=[chunk],
            embeddings=[embedding],
            ids=[str(i)]
        )

In [12]:
def retrieve(query: str, top_k: int) -> List[str]:
    query_embedding = embed_chunk(query)
    results = chromadb_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results['documents'][0]


In [13]:
def rerank(query: str, retrieved_chunks: List[str], top_k: int) -> List[str]:
    cross_encoder = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')
    pairs = [(query, chunk) for chunk in retrieved_chunks]
    scores = cross_encoder.predict(pairs)

    scored_chunks = list(zip(retrieved_chunks, scores))
    scored_chunks.sort(key=lambda x: x[1], reverse=True)

    return [chunk for chunk, _ in scored_chunks][:top_k]


In [14]:
def generate(query: str, chunks: List[str]) -> str:
    prompt = f"""你是一位知识助手，请根据用户的问题和下列片段生成准确的回答。

用户问题: {query}

相关片段:
{"\n\n".join(chunks)}

请基于上述内容作答，不要编造信息。"""

    response = google_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    return response.text

In [15]:
def interact_answer():
    doc = input("Please input doc path.")
    query = input("Please input your query.")
   
    chunks = split_into_chunks(doc)
    embeddings = [embed_chunk(chunk) for chunk in chunks]
    save_embeddings(chunks, embeddings)
    retrieved_chunks = retrieve(query, math.ceil(len(chunks)*0.5))
    reranked_chunks = rerank(query, retrieved_chunks, math.ceil(len(chunks)*0.5))

    answer = generate(query, reranked_chunks)
    print(answer)
    

In [23]:
if __name__ == '__main__':
    interact_answer()

Please input doc path. C:\Users\shuyu\Desktop\基于多指标优化的 NIPT 最佳时点选择与胎儿异常判定研究.pdf
Please input your query. 问题一用了什么方法


根据提供的片段，“问题一”最可能指的是摘要中描述的“方案二”（ֻ၂໙）所采用的方法。该方案主要运用了以下方法：

1.  **数据预处理：** 包括多维度数据转换、有效性检验、缺失值插补、标准化、中位数处理和异常值处理等数据处理步骤。
2.  **统计建模：**
    *   **Logit回归模型：** 用于分析Y染色体片段浓度与相关因素的关系。
    *   **B样条函数：** 引入B样条函数来处理孕周和BMI的非线性关系。
    *   **广义线性多因素逻辑回归模型：** 用于构建预测模型。
3.  **模型评估与诊断：**
    *   **F检验：** 用于评估模型的整体显著性。
    *   **多重共线性诊断：** 用于检测解释变量之间的多重共线性。
    *   **正态性检验：** 用于检验模型残差的正态性。
